In [ ]:
!pip install pathway bokeh --quiet

# 🚗 Model 1: Baseline Linear Pricing Model
This notebook calculates real-time parking prices using a simple linear logic based on occupancy and capacity.


In [ ]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/SakshiSan/dynamic-pricing-parking/main/dataset.csv')
display(df)

### 🕒 Combine Date & Time Columns
We create a new timestamp column by combining 'LastUpdatedDate' and 'LastUpdatedTime'.


In [ ]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

df = df.sort_values('Timestamp').reset_index(drop=True)

### 💾 Save Selected Columns for Pathway Stream Simulation


In [ ]:
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)

### 🏗️ Define Schema
I defined the structure of incoming streaming data using Pathway’s Schema class.


In [ ]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int

### 🔁 Load Streaming Data
I simulated real-time data ingestion using `replay_csv()` at a defined rate of 1000 rows/sec.


In [ ]:
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

### 🕒 Convert and Extract Time Information
I parse the timestamp into a datetime object and extract the day for daily analysis.


In [ ]:
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# Step 2: Making a simple pricing function

### 📊 Daily Aggregation & Price Calculation
We calculate the daily price using:


In [ ]:
import datetime

delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
    )
    .with_columns(
         price=10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)

# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

### 📉 Bokeh Visualization for Final Price


In [ ]:
pn.extension()
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",
    )

    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.scatter("t", "price", source=source, size=6, color="red")
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()

In [ ]:
%%capture --no-display
pw.run()


## 📝 Conclusion  
This project demonstrates a dynamic pricing model using vehicle occupancy and capacity data to calculate parking demand and determine a fair price.  
Using Pathway’s real-time stream processing and Bokeh for visualization, we can see how the demand and final price change over time.  
This approach helps in building intelligent systems for smart parking.
